In [1]:
# For output clarity 
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np 
import re

In [3]:
#Importing clearn sharks data--rows confirmed attack 
# From kaggle https://www.kaggle.com/datasets/thedevastator/shark-attacks-the-risks-of-coastal-water-activit
# FromShark Research Institute's website https://www.sharkattackfile.net/incidentlog.htm
file_path = "shark_bite_confirmed.csv"
confirmed_attack_df = pd.read_csv(file_path)
confirmed_attack_df.columns

Index(['index', 'Case Number', 'Date', 'Year', 'Type', 'Country', 'Area',
       'Location', 'Activity', 'Name', 'Unnamed: 9', 'Age', 'Injury',
       'Fatal (Y/N)', 'Time', 'Species'],
      dtype='object')

In [4]:
columns = ["Case Number", "Name", "Type", "Injury","Fatal (Y/N)"]
fatality_df = confirmed_attack_df.loc[:, columns]
fatality_df.rename(columns = {'Fatal (Y/N)': 'Fatal'}, inplace = True)
fatality_df['Fatal'].fillna('U', inplace = True)
fatality_df

,Case Number,Name,Type,Injury,Fatal
0,2020.02.05,NaN,Unprovoked,"No injury, but paddleboard bitten",N
1,2020.01.30.R,Ana Bruna Avila,Provoked,PROVOKED INCIDENT Scratches to left wrist,N
2,2020.01.17,Will Schroeter,Unprovoked,Laceration ot left ankle and foot,N
3,2020.01.16,Jordan King,Unprovoked,Minor injury to lower leg,N
4,2020.01.13,Samuel Horne,Unprovoked,Lacerations to foot,N
...,...,...,...,...,...
5930,ND.0005,male,Unprovoked,FATAL,Y
5931,ND.0004,Ahmun,Unprovoked,FATAL,Y
5932,ND.0003,Coast Guard personnel,Unprovoked,FATAL,Y
5933,ND.0002,Jules Patterson,Unprovoked,FATAL,Y


In [5]:
list(fatality_df["Fatal"].unique())

['N', 'Y', 'U', 'F', 'M', 'UNKNOWN', '2017', 'Unknown', 'y']

In [6]:
fatality_df.loc[fatality_df['Fatal'] == 'y', 'Fatal'] = 'Y'
fatality_df.loc[fatality_df['Fatal'] == 'F', 'Fatal'] = 'Y'
fatality_df.loc[fatality_df['Fatal'] == 'UNKNOWN', 'Fatal'] = 'U'
fatality_df.loc[fatality_df['Fatal'] == 'Unknown', 'Fatal'] = 'U'

In [7]:
M_df = fatality_df.loc[fatality_df['Fatal'] == 'M']
M_df

,Case Number,Name,Type,Injury,Fatal
416,2016.04.18.b,Hoata Iotua,Provoked,Laceration to knee by speared shark PROVOKED I...,M


In [8]:
fatality_df.loc[fatality_df['Fatal'] == 'M', 'Fatal'] = 'N'

In [9]:
date_df = fatality_df.loc[fatality_df['Fatal'] == '2017']
date_df

,Case Number,Name,Type,Injury,Fatal
876,2012.06.10,Giorgio Zara,Provoked,Lower left leg injured PROVOKED ACCIDENT,2017


In [10]:
fatality_df.loc[fatality_df['Fatal'] == '2017', 'Fatal'] = 'N'

In [11]:
list(fatality_df["Fatal"].unique())

['N', 'Y', 'U']

In [14]:
fatality_df['Injury'].fillna('U', inplace = True)

In [17]:
U = fatality_df.loc[(fatality_df['Fatal'] == 'U') & (fatality_df['Injury'] == 'U')]


In [18]:
fatality_df.drop(U.index, inplace=True)

In [20]:
fatality_df.loc[fatality_df['Fatal'] == 'U']

,Case Number,Name,Type,Injury,Fatal
15,2019.12.20,NaN,Provoked,PROVOKED INCIDENT,U
31,2019.10.21,Emmanuelle B,Unprovoked,Severe injuries to arms and chest,U
57,2019.07.28,female,Unprovoked,Lacerations to lower leg,U
61,2019.07.18,NaN,Unconfirmed,Unknown / Unconfirmed,U
136,2018.07.16.R,Laura Dean,Invalid,Single laceration to right thigh inconsistent ...,U
...,...,...,...,...,...
5746,1826.08.15.R,child,Unprovoked,FATAL,U
5761,1808.06.01.R,NaN,Sea Disaster,Probable drowning & scavenging,U
5764,1806.04.24,Crew of the schooner Mary,Sea Disaster,Shark scavenged on the dead sailors,U
5794,1723.00.00,Philip Ashton,Unprovoked,Struck on thigh,U


In [26]:
fatality_df.loc[fatality_df['Injury'] == 'FATAL', 'Fatal'] = 'Y'
fatality_df.loc[fatality_df['Fatal'] == 'U']

,Case Number,Name,Type,Injury,Fatal
15,2019.12.20,NaN,Provoked,PROVOKED INCIDENT,U
31,2019.10.21,Emmanuelle B,Unprovoked,Severe injuries to arms and chest,U
57,2019.07.28,female,Unprovoked,Lacerations to lower leg,U
61,2019.07.18,NaN,Unconfirmed,Unknown / Unconfirmed,U
136,2018.07.16.R,Laura Dean,Invalid,Single laceration to right thigh inconsistent ...,U
137,2018.07.15,Fidelino Gómez Bernard,Invalid,"Autopsy confirmed death by drowning, shark bit...",U
171,2018.04.15.c,Pablo de Melo,Unprovoked,"Multiple severe injuries to arms and leg, leg ...",U
226,2017.09.15.b,Wayon Love,Invalid,"FATAL, but death was probably due to drowning",U
248,2017.07.29.c,Johnny Simatacolos,Invalid,Abrasions and cuts to sole of foot,U
322,2017.01.08.R,Kerry Daniel,Invalid,"No attack, shark made a threat display",U


In [27]:
Invalid = fatality_df.loc[fatality_df['Type'] == 'Invalid']

In [28]:
fatality_df.drop(Invalid.index, inplace=True)

In [29]:
fatality_df.loc[fatality_df['Fatal'] == 'U']

,Case Number,Name,Type,Injury,Fatal
15,2019.12.20,NaN,Provoked,PROVOKED INCIDENT,U
31,2019.10.21,Emmanuelle B,Unprovoked,Severe injuries to arms and chest,U
57,2019.07.28,female,Unprovoked,Lacerations to lower leg,U
61,2019.07.18,NaN,Unconfirmed,Unknown / Unconfirmed,U
171,2018.04.15.c,Pablo de Melo,Unprovoked,"Multiple severe injuries to arms and leg, leg ...",U
400,2016.06.02.b,Waade Madigan and Dr Seyong Kim,Unprovoked,"No injury, but sharks repeatedly hit their fin...",U
504,2015.07.27,male,NaN,Injury to hand,U
727,2013.09.08,female,Unprovoked,No details,U
1348,2008.01.30,NaN,Provoked,"No injury, captive shark attempted to bite div...",U
2241,1997.02.21,Gersome Perreno,Unprovoked,No details,U


In [30]:
fatality_df.loc[fatality_df['Injury'] == 'No injury', 'Fatal'] = 'N'

In [31]:
fatality_df.loc[fatality_df['Fatal'] == 'U']

,Case Number,Name,Type,Injury,Fatal
15,2019.12.20,NaN,Provoked,PROVOKED INCIDENT,U
31,2019.10.21,Emmanuelle B,Unprovoked,Severe injuries to arms and chest,U
57,2019.07.28,female,Unprovoked,Lacerations to lower leg,U
61,2019.07.18,NaN,Unconfirmed,Unknown / Unconfirmed,U
171,2018.04.15.c,Pablo de Melo,Unprovoked,"Multiple severe injuries to arms and leg, leg ...",U
400,2016.06.02.b,Waade Madigan and Dr Seyong Kim,Unprovoked,"No injury, but sharks repeatedly hit their fin...",U
504,2015.07.27,male,NaN,Injury to hand,U
727,2013.09.08,female,Unprovoked,No details,U
1348,2008.01.30,NaN,Provoked,"No injury, captive shark attempted to bite div...",U
2241,1997.02.21,Gersome Perreno,Unprovoked,No details,U


In [37]:
fatality_df.loc[fatality_df['Case Number'] == '2008.01.30', 'Fatal'] = 'N'
fatality_df.loc[fatality_df['Case Number'] == '1989.12.03', 'Fatal'] = 'N'
fatality_df.loc[fatality_df['Case Number'] == '1958.12.28', 'Fatal'] = 'N'
fatality_df.loc[fatality_df['Case Number'] == '1954.07.27', 'Fatal'] = 'N'
fatality_df.loc[fatality_df['Case Number'] == '1894.06.29', 'Fatal'] = 'N'
fatality_df.loc[fatality_df['Case Number'] == '1967.09.13', 'Fatal'] = 'N'
fatality_df.loc[fatality_df['Case Number'] == '2016.06.02.b', 'Fatal'] = 'N'
fatality_df.loc[fatality_df['Case Number'] == '2018.04.15.c', 'Fatal'] = 'N'
fatality_df.loc[fatality_df['Case Number'] == '2019.07.28', 'Fatal'] = 'N'
fatality_df.loc[fatality_df['Case Number'] == '2019.10.21', 'Fatal'] = 'N'

fatality_df.loc[fatality_df['Case Number'] == '1723.00.00', 'Fatal'] = 'N'
fatality_df.loc[fatality_df['Case Number'] == '1981.10.17', 'Fatal'] = 'N'
fatality_df.loc[fatality_df['Case Number'] == '1982.00.00.c', 'Fatal'] = 'N'
fatality_df.loc[fatality_df['Case Number'] == '1983.04.19', 'Fatal'] = 'N'
fatality_df.loc[fatality_df['Case Number'] == '1995.12.18', 'Fatal'] = 'N'
fatality_df.loc[fatality_df['Case Number'] == '1996.04.28.b', 'Fatal'] = 'Y'
fatality_df.loc[fatality_df['Case Number'] == '1996.07.14', 'Fatal'] = 'N'

fatality_df.loc[fatality_df['Case Number'] == '1997.02.21', 'Fatal'] = 'N'
fatality_df.loc[fatality_df['Case Number'] == '2015.07.27', 'Fatal'] = 'N'
fatality_df.loc[fatality_df['Case Number'] == '1957.02.02', 'Fatal'] = 'N'

In [39]:
Unknown = fatality_df.loc[fatality_df['Fatal'] == 'U']

In [40]:
fatality_df.drop(Unknown.index, inplace=True)

In [41]:
list(fatality_df["Fatal"].unique())

['N', 'Y']

In [ ]:
country_df.to_csv("fatality.csv", index=False)